<a href="https://colab.research.google.com/github/lofel888/ML_code_projects/blob/main/Collision_avoidance_Kaggle_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подготовка данных

In [ ]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     --------------------------------- ------ 41.0/48.7 kB 1.9 MB/s eta 0:00:01
     --------------------------------- ------ 41.0/48.7 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 409.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.3 MB 3.3 MB/s eta 0:01:59
   ---------------------------------------- 0.5/390.3 MB 4.7 MB/s eta 0:01:24
   ---------------------------------------- 0.6/390.3 MB 5.8 MB/s eta 0:01:08
   ---------------------------------------- 0.6/390.3 MB 5.8 MB/s eta 0:01:08
   ---------------------------------------- 1.2/390.3 MB 6.0 MB/s eta 0:01:06
   ---------------------------------------- 1.5/390.3 MB 5.4 MB/s eta 0:01:12
   ---------------------------------------- 2.2/390.3 MB 7.0 MB/s eta 0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
train_data= pd.read_csv('Downloads/train_data.csv')

,event_id,time_to_tca,mission_id,risk,max_risk_estimate,max_risk_scaling,miss_distance,relative_speed,relative_position_r,relative_position_t,...,t_sigma_rdot,c_sigma_rdot,t_sigma_tdot,c_sigma_tdot,t_sigma_ndot,c_sigma_ndot,F10,F3M,SSN,AP
0,0,1.566798,5,-10.204955,-7.834756,8.602101,14923.0,13792.0,453.8,5976.6,...,0.147350,58.272095,0.004092,0.165044,0.002987,0.386462,89.0,83.0,42.0,11.0
1,0,1.207494,5,-10.355758,-7.848937,8.956374,14544.0,13792.0,474.3,5821.2,...,0.059672,57.966413,0.003753,0.164383,0.002933,0.386393,89.0,83.0,42.0,11.0
2,0,0.952193,5,-10.345631,-7.847406,8.932195,14475.0,13792.0,474.6,5796.2,...,0.039258,57.907599,0.003576,0.164352,0.002967,0.386381,89.0,83.0,42.0,11.0
3,0,0.579669,5,-10.337809,-7.845880,8.913444,14579.0,13792.0,472.7,5838.9,...,0.022066,57.993905,0.003298,0.164309,0.002918,0.386400,89.0,83.0,40.0,14.0
4,0,0.257806,5,-10.391260,-7.852942,9.036838,14510.0,13792.0,478.7,5811.1,...,0.015075,57.946717,0.003670,0.164172,0.003220,0.386388,89.0,83.0,40.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162629,13153,3.029751,1,-7.108630,-5.142668,7.561579,2307.0,14976.0,-472.1,-159.9,...,0.071060,48.480202,0.011479,0.145746,0.002764,0.231577,81.0,84.0,48.0,18.0
162630,13153,2.799253,1,-7.070070,-5.137869,7.471857,2254.0,14976.0,-469.7,-163.3,...,0.063959,48.565132,0.009935,0.145729,0.002678,0.231583,78.0,84.0,49.0,19.0
162631,13153,2.385399,1,-7.066209,-5.137034,7.463470,2259.0,14976.0,-469.3,-158.8,...,0.059657,48.547317,0.009491,0.145745,0.002632,0.231582,78.0,84.0,49.0,19.0
162632,13153,2.043339,1,-7.028307,-5.131297,7.378397,2257.0,14976.0,-465.2,-160.3,...,0.039351,48.540602,0.002612,0.145734,0.002631,0.231583,78.0,84.0,49.0,19.0


In [ ]:
# Count the occurrences of each event_id
event_id_counts = train_data['event_id'].value_counts()
# Получаем последние строки для каждого event_id
last_rows = train_data.groupby('event_id').tail(1)
# print(last_rows)
# Фильтруем строки, где значение risk >= -6
filtered_rows = last_rows[last_rows['risk'] >= -6]

# Считаем количество таких групп
count_of_groups = len(filtered_rows)

# Выводим результат
print(f"Количество групп, где в последних строках risk >= -6: {count_of_groups}")

# Filter out rows where event_id is unique
train_data_filtered = train_data[train_data['event_id'].isin(event_id_counts[event_id_counts > 1].index)]


Количество групп, где в последних строках risk >= -6: 365


,event_id,time_to_tca,mission_id,risk,max_risk_estimate,max_risk_scaling,miss_distance,relative_speed,relative_position_r,relative_position_t,...,t_sigma_rdot,c_sigma_rdot,t_sigma_tdot,c_sigma_tdot,t_sigma_ndot,c_sigma_ndot,F10,F3M,SSN,AP
0,0,1.566798,5,-10.204955,-7.834756,8.602101,14923.0,13792.0,453.8,5976.6,...,0.147350,58.272095,0.004092,0.165044,0.002987,0.386462,89.0,83.0,42.0,11.0
1,0,1.207494,5,-10.355758,-7.848937,8.956374,14544.0,13792.0,474.3,5821.2,...,0.059672,57.966413,0.003753,0.164383,0.002933,0.386393,89.0,83.0,42.0,11.0
2,0,0.952193,5,-10.345631,-7.847406,8.932195,14475.0,13792.0,474.6,5796.2,...,0.039258,57.907599,0.003576,0.164352,0.002967,0.386381,89.0,83.0,42.0,11.0
3,0,0.579669,5,-10.337809,-7.845880,8.913444,14579.0,13792.0,472.7,5838.9,...,0.022066,57.993905,0.003298,0.164309,0.002918,0.386400,89.0,83.0,40.0,14.0
4,0,0.257806,5,-10.391260,-7.852942,9.036838,14510.0,13792.0,478.7,5811.1,...,0.015075,57.946717,0.003670,0.164172,0.003220,0.386388,89.0,83.0,40.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162629,13153,3.029751,1,-7.108630,-5.142668,7.561579,2307.0,14976.0,-472.1,-159.9,...,0.071060,48.480202,0.011479,0.145746,0.002764,0.231577,81.0,84.0,48.0,18.0
162630,13153,2.799253,1,-7.070070,-5.137869,7.471857,2254.0,14976.0,-469.7,-163.3,...,0.063959,48.565132,0.009935,0.145729,0.002678,0.231583,78.0,84.0,49.0,19.0
162631,13153,2.385399,1,-7.066209,-5.137034,7.463470,2259.0,14976.0,-469.3,-158.8,...,0.059657,48.547317,0.009491,0.145745,0.002632,0.231582,78.0,84.0,49.0,19.0
162632,13153,2.043339,1,-7.028307,-5.131297,7.378397,2257.0,14976.0,-465.2,-160.3,...,0.039351,48.540602,0.002612,0.145734,0.002631,0.231583,78.0,84.0,49.0,19.0


In [ ]:
# Проверка всех условий одновременно
has_valid_tca_combined = train_data_filtered.groupby('event_id')['time_to_tca'].transform(
    lambda x: (x <= 2).any() and (x > 2).any() and (x < 1).any()
)

# Фильтрация строк, соответствующих объединённым условиям
train_data_filtered_1 = train_data_filtered[has_valid_tca_combined]

# Добавление столбца high_low_risk_CDM и заполнение значениями 1, если выполняются условия
train_data_filtered_1 = train_data_filtered_1.assign(high_low_risk_CDM=np.where( (train_data_filtered_1['risk'] >= -6), 1, 0))
# Подсчет количества строк с high_low_risk_CDM = 1
high_low_risk_CDM_counts = train_data_filtered_1['high_low_risk_CDM'].value_counts()

# Вычисление процента строк с high_low_risk_CDM = 1
percentage_high_low_risk_CDM_1 = (high_low_risk_CDM_counts[1] / high_low_risk_CDM_counts.sum()) * 100

# Вывод результата
print(f"Процент high-risk CDMs: {percentage_high_low_risk_CDM_1:.2f}%")
print(f"Процент low-risk CDMs: {(100-percentage_high_low_risk_CDM_1):.2f}%")
# Вывод отфильтрованных данных
train_data_filtered_1
# # Фильтрация групп, где в последней строке high_low_risk_CDM == 1
# groups_with_high_risk = [
#     group for group in event_id_groups if group.iloc[-1]['high_low_risk_CDM'] == 1
# ]
# print(len(groups_with_high_risk))
# # Получаем последние строки для каждого event_id
# last_rows = train_data_filtered_1.groupby('event_id').tail(1)

# # Фильтруем строки, где значение risk <= -6
# filtered_rows = last_rows[last_rows['risk'] >= -6]

# # Считаем количество таких групп
# count_of_groups = len(filtered_rows)

# # Выводим результат
# print(f"Количество групп, где в последних строках risk <= -6: {count_of_groups}")


Процент high-risk CDMs: 6.03%
Процент low-risk CDMs: 93.97%


,event_id,time_to_tca,mission_id,risk,max_risk_estimate,max_risk_scaling,miss_distance,relative_speed,relative_position_r,relative_position_t,...,c_sigma_rdot,t_sigma_tdot,c_sigma_tdot,t_sigma_ndot,c_sigma_ndot,F10,F3M,SSN,AP,high_low_risk_CDM
9,2,6.983474,2,-10.816161,-6.601713,13.293159,22902.0,14348.0,-1157.6,-6306.2,...,39.695541,0.009370,0.269965,0.003886,0.339406,73.0,77.0,27.0,4.0,0
10,2,6.691611,2,-10.850473,-6.603452,13.374242,22966.0,14348.0,-1161.1,-6330.2,...,39.617319,0.009373,0.269926,0.003939,0.339393,73.0,77.0,27.0,4.0,0
11,2,6.269979,2,-30.000000,-6.217958,426.808532,18785.0,14347.0,-698.8,-5176.4,...,2.616749,0.010256,0.050424,0.003963,0.193127,71.0,77.0,23.0,8.0,0
12,2,6.042352,2,-30.000000,-6.271078,181.496778,18842.0,14347.0,-700.0,-5192.1,...,6.985796,0.009402,0.051577,0.003920,0.193324,71.0,77.0,23.0,8.0,0
13,2,5.711716,2,-30.000000,-6.277448,187.525360,19015.0,14347.0,-709.9,-5242.1,...,6.878515,0.008628,0.051522,0.004023,0.193324,71.0,77.0,23.0,8.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162616,13152,1.484446,15,-30.000000,-7.111427,1510.933859,24980.0,1156.0,-1158.4,-24911.2,...,0.528828,0.001381,0.041548,0.001573,0.102720,71.0,72.0,16.0,1.0,0
162617,13152,1.075997,15,-30.000000,-7.113679,1536.096361,25023.0,1156.0,-1157.4,-24954.3,...,0.522801,0.001200,0.041534,0.001511,0.102721,71.0,72.0,16.0,1.0,0
162618,13152,0.677213,15,-30.000000,-7.114243,1546.486178,25032.0,1156.0,-1157.6,-24963.0,...,0.520203,0.001159,0.041526,0.001703,0.102722,71.0,72.0,16.0,1.0,0
162619,13152,0.446067,15,-30.000000,-7.394264,19867.782213,25214.0,1156.0,-1204.9,-25143.6,...,0.093888,0.001167,0.023695,0.001678,0.058784,71.0,72.0,12.0,1.0,0


In [ ]:
#TREES
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# Функция для фильтрации данных в каждой группе
def filter_event_data(group):
    # Удаляем все строки, где time_to_tca < 2
    filtered_group = group[group['time_to_tca'] >= 2]
    # Сохраняем последнюю строку, даже если time_to_tca < 2
    last_row = group.iloc[-1:]  # Последняя строка группы
    # Объединяем отфильтрованные данные с последней строкой
    return pd.concat([filtered_group, last_row]).drop_duplicates()

# Применение функции к каждому event_id в train_data_1
main_data_set = train_data_filtered_1.groupby('event_id').apply(filter_event_data).reset_index(drop=True)
print(len(main_data_set))
# Сопоставление значений из c_object_type в числовые категории
object_type_mapping = {
    'DEBRIS': 0,
    'UNKNOWN': 1,
    'PAYLOAD': 2,
    'ROCKET BODY': 3,
    'TBA': 4
}

# Создание нового столбца с использованием маппинга
main_data_set['c_object_type_num'] = main_data_set['c_object_type'].map(object_type_mapping)
# Вычисляем средние значения для столбцов
mean_c_position_covariance_det = main_data_set['c_position_covariance_det'].mean()
mean_t_position_covariance_det = main_data_set['t_position_covariance_det'].mean()
# Преобразуем значения в столбцах
main_data_set['c_position_covariance_det_normalized'] = main_data_set['c_position_covariance_det'].apply(
    lambda x: 0 if (x / mean_c_position_covariance_det) < 1e-38 else (1 if (x / mean_c_position_covariance_det) > 1e38 else x / mean_c_position_covariance_det)
)

main_data_set['t_position_covariance_det_normalized'] = main_data_set['t_position_covariance_det'].apply(
    lambda x: 0 if (x / mean_t_position_covariance_det) < 1e-38 else (1 if (x / mean_t_position_covariance_det) > 1e38 else x / mean_t_position_covariance_det)
)
# Разделение данных на группы по event_id
event_id_groups = [group for _, group in main_data_set.groupby('event_id')]

# Фильтрация групп, где в последней строке high_low_risk_CDM == 1
groups_with_high_risk = [
    group for group in event_id_groups if group.iloc[-1]['high_low_risk_CDM'] == 1
]
# print(len(groups_with_high_risk))
# Проверяем, достаточно ли групп с high_low_risk_CDM == 1 для тестовой выборки
if len(groups_with_high_risk) < 25:
    raise ValueError("Недостаточно групп с high_low_risk_CDM == 1 для тестовой выборки")

# Определяем количество групп для тестовой выборки
total_test_size = int(0.2 * len(event_id_groups))
test_high_risk_size = 20

# Выделяем группы с high_risk для тестовой выборки
test_high_risk_groups = groups_with_high_risk[:test_high_risk_size]

# Получаем список event_id из test_high_risk_groups
test_high_risk_event_ids = [group['event_id'].iloc[0] for group in test_high_risk_groups]

# Фильтруем оставшиеся группы, исключая группы с event_id из test_high_risk_groups
remaining_groups = [
    group for group in event_id_groups if group['event_id'].iloc[0] not in test_high_risk_event_ids
]

# Определяем количество оставшихся групп для тестовой выборки
remaining_test_size = total_test_size - len(test_high_risk_groups)

# Разделяем оставшиеся группы на тренировочную и тестовую выборки
train_groups, remaining_test_groups = train_test_split(
    remaining_groups, test_size=remaining_test_size / len(remaining_groups), random_state=42
)

# Объединяем тестовые группы
test_groups = test_high_risk_groups + remaining_test_groups

# Объединение групп обратно в DataFrame
train_data_1 = pd.concat(train_groups)
test_data_1 = pd.concat(test_groups)

# Проверяем итоговые размеры выборок
print(f"Всего групп: {len(event_id_groups)}")
print(f"Размер обучающей выборки: {len(train_groups)} групп ({len(train_groups)/len(event_id_groups):.2%})")
print(f"Размер тестовой выборки: {len(test_groups)} групп ({len(test_groups)/len(event_id_groups):.2%})")
print(f"Групп с high_low_risk_CDM == 1 в тестовой выборке: {len(test_high_risk_groups)}")
len(test_data_1)
# # Удаление двух последних столбцов из обеих выборок
# train_data_1 = train_data_1.iloc[:, :-2]  # Удаляем два последних столбца
# test_data_1 = test_data_1.iloc[:, :-2]  # Удаляем два последних столбца

C:\Users\lofel\AppData\Local\Temp\ipykernel_18996\3167059151.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  main_data_set = train_data_filtered_1.groupby('event_id').apply(filter_event_data).reset_index(drop=True)


101000
Всего групп: 8293
Размер обучающей выборки: 6635 групп (80.01%)
Размер тестовой выборки: 1658 групп (19.99%)
Групп с high_low_risk_CDM == 1 в тестовой выборке: 20


20257

In [ ]:
# # Функция для фильтрации данных в каждой группе
# def filter_event_data(group):
#     # Удаляем все строки, где time_to_tca < 2
#     filtered_group = group[group['time_to_tca'] >= 2]
#     # Сохраняем последнюю строку, даже если time_to_tca < 2
#     last_row = group.iloc[-1:]  # Последняя строка группы
#     # Объединяем отфильтрованные данные с последней строкой
#     return pd.concat([filtered_group, last_row]).drop_duplicates()

# # Применение функции к каждому event_id в train_data_1
# train_data_filtered_12 = train_data_1.groupby('event_id').apply(filter_event_data).reset_index(drop=True)
# test_data_filtered_12 = test_data_1.groupby('event_id').apply(filter_event_data).reset_index(drop=True)
# Удаляем все данные из последней строки каждого event_id в test_data_filtered, кроме event_id
def clean_last_row(df):
    # Создаем копию DataFrame, чтобы не изменять исходные данные
    df_copy = df.copy()
    # Группируем по event_id
    df_copy['is_last'] = df_copy.groupby('event_id').cumcount(ascending=False) == 0
    # Устанавливаем все значения в последней строке, кроме event_id, в NaN
    df_copy.loc[df_copy['is_last'], df_copy.columns != 'event_id'] = None
    # Удаляем временный столбец 'is_last'
    df_copy.drop(columns=['is_last'], inplace=True)
    return df_copy

# Применяем функцию к test_data_1
test_data_1 = clean_last_row(test_data_1)
# Вывод размеров обучающей и тестовой выборок
print(f"Размер обучающей выборки: {len(train_data_1)}")
print(f"Размер тестовой выборки после фильтрации: {len(test_data_1)}")
train_data_1.head(50)
# test_data_1.head(50)

Размер обучающей выборки: 80743
Размер тестовой выборки после фильтрации: 20257


C:\Users\lofel\AppData\Local\Temp\ipykernel_18996\4168270262.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_copy.loc[df_copy['is_last'], df_copy.columns != 'event_id'] = None


,event_id,time_to_tca,mission_id,risk,max_risk_estimate,max_risk_scaling,miss_distance,relative_speed,relative_position_r,relative_position_t,...,t_sigma_ndot,c_sigma_ndot,F10,F3M,SSN,AP,high_low_risk_CDM,c_object_type_num,c_position_covariance_det_normalized,t_position_covariance_det_normalized
57657,7488,2.463465,1,-30.000000,-5.181115,1.939153e+03,8176.0,8191.0,-281.6,6848.1,...,0.002742,0.017557,78.0,77.0,30.0,9.0,0,0,5.351482e-37,0.000000e+00
57658,7488,2.200409,1,-30.000000,-4.913284,2.526501e+02,8431.0,8191.0,-285.9,7061.2,...,0.002731,0.017572,78.0,77.0,30.0,9.0,0,0,9.011747e-36,0.000000e+00
57659,7488,0.160409,1,-30.000000,-5.359817,2.112044e+04,8056.0,8191.0,-316.3,6745.6,...,0.002453,0.011300,77.0,77.0,35.0,6.0,0,0,0.000000e+00,0.000000e+00
41904,5399,4.755639,7,-30.000000,-7.584359,1.649816e+02,17045.0,13558.0,483.8,7676.0,...,0.004718,0.116762,80.0,82.0,29.0,11.0,0,1,9.778144e-30,5.881397e-37
41905,5399,4.414599,7,-30.000000,-7.596193,1.570871e+02,29552.0,13558.0,286.8,13310.3,...,0.004695,0.122685,80.0,82.0,29.0,11.0,0,1,1.129738e-29,4.627070e-37
41906,5399,4.084568,7,-30.000000,-7.620332,1.982047e+02,53079.0,13558.0,-167.6,23913.1,...,0.004440,0.108750,75.0,82.0,11.0,17.0,0,1,4.965855e-30,4.416244e-37
41907,5399,3.717039,7,-30.000000,-7.474696,2.122324e+02,24206.0,13558.0,286.6,10904.9,...,0.004115,0.084223,75.0,82.0,11.0,17.0,0,1,1.917693e-30,1.754636e-37
41908,5399,3.386796,7,-30.000000,-7.413525,2.057298e+02,11212.0,13558.0,483.0,5043.2,...,0.004042,0.078145,75.0,82.0,23.0,14.0,0,1,1.548545e-30,8.894793e-38
41909,5399,3.061254,7,-30.000000,-7.292089,1.410700e+02,29529.0,13558.0,-13.3,13303.7,...,0.003693,0.086573,75.0,82.0,23.0,14.0,0,1,1.343170e-29,6.592219e-38
41910,5399,2.721889,7,-30.000000,-7.046047,9.705739e+01,11044.0,13558.0,271.6,4970.8,...,0.003777,0.076746,75.0,82.0,23.0,14.0,0,1,9.234759e-30,5.144917e-38


In [ ]:
# Получение списка названий столбцов из train_data_filtered
df_copy = [f'{col}' for col in train_data_1.columns]
df_copy

['event_id',
 'time_to_tca',
 'mission_id',
 'risk',
 'max_risk_estimate',
 'max_risk_scaling',
 'miss_distance',
 'relative_speed',
 'relative_position_r',
 'relative_position_t',
 'relative_position_n',
 'relative_velocity_r',
 'relative_velocity_t',
 'relative_velocity_n',
 't_time_lastob_start',
 't_time_lastob_end',
 't_recommended_od_span',
 't_actual_od_span',
 't_obs_available',
 't_obs_used',
 't_residuals_accepted',
 't_weighted_rms',
 't_rcs_estimate',
 't_cd_area_over_mass',
 't_cr_area_over_mass',
 't_sedr',
 't_j2k_sma',
 't_j2k_ecc',
 't_j2k_inc',
 't_ct_r',
 't_cn_r',
 't_cn_t',
 't_crdot_r',
 't_crdot_t',
 't_crdot_n',
 't_ctdot_r',
 't_ctdot_t',
 't_ctdot_n',
 't_ctdot_rdot',
 't_cndot_r',
 't_cndot_t',
 't_cndot_n',
 't_cndot_rdot',
 't_cndot_tdot',
 'c_object_type',
 'c_time_lastob_start',
 'c_time_lastob_end',
 'c_recommended_od_span',
 'c_actual_od_span',
 'c_obs_available',
 'c_obs_used',
 'c_residuals_accepted',
 'c_weighted_rms',
 'c_rcs_estimate',
 'c_cd_area_

In [ ]:
#Regressor
def create_features(group):
    # Убедимся, что работаем с DataFrame и преобразуем столбцы в числовой тип
    group = group.copy()  # Создаем копию группы, чтобы избежать изменений в оригинальных данных
    columns_to_convert = [
        'time_to_tca', 'risk', 'max_risk_estimate', 'max_risk_scaling', 'mahalanobis_distance',
        'miss_distance', 'c_obs_used', 'relative_speed', 'relative_position_r', 'relative_position_t',
        'relative_position_n', 'relative_velocity_r', 'relative_velocity_t', 'relative_velocity_n',
        't_time_lastob_start', 't_time_lastob_end', 't_recommended_od_span', 't_actual_od_span',
        't_obs_available', 't_obs_used', 't_residuals_accepted', 't_weighted_rms', 't_rcs_estimate',
        't_cd_area_over_mass', 't_cr_area_over_mass', 't_sedr', 't_j2k_sma', 't_j2k_ecc', 't_j2k_inc',
        't_ct_r', 't_cn_r', 't_cn_t', 't_crdot_r', 't_crdot_t', 't_crdot_n', 't_ctdot_r', 't_ctdot_t',
        't_ctdot_n', 't_ctdot_rdot', 't_cndot_r', 't_cndot_t', 't_cndot_n', 't_cndot_rdot',
        't_cndot_tdot', 'c_time_lastob_start', 'c_time_lastob_end',
        'c_recommended_od_span','t_position_covariance_det_normalized','c_position_covariance_det_normalized',
        'c_actual_od_span', 'c_obs_available',
        'c_residuals_accepted', 'c_weighted_rms', 'c_rcs_estimate', 'c_cd_area_over_mass',
        'c_cr_area_over_mass', 'c_sedr', 'c_j2k_sma', 'c_j2k_ecc', 'c_j2k_inc', 'c_ct_r', 'c_cn_r',
        'c_cn_t', 'c_crdot_r', 'c_crdot_t', 'c_crdot_n', 'c_ctdot_r', 'c_ctdot_t', 'c_ctdot_n',
        'c_ctdot_rdot', 'c_cndot_r', 'c_cndot_t', 'c_cndot_n', 'c_cndot_rdot', 'c_cndot_tdot', 't_span',
        'c_span', 't_h_apo', 't_h_per', 'c_h_apo', 'c_h_per', 'geocentric_latitude', 'azimuth',
        'elevation', 't_sigma_r', 'c_sigma_r', 't_sigma_t', 'c_sigma_t', 't_sigma_n', 'c_sigma_n', 't_sigma_rdot',
        'c_sigma_rdot', 't_sigma_tdot', 'c_sigma_tdot', 't_sigma_ndot', 'c_sigma_ndot', 'F10', 'F3M',
        'SSN', 'AP', 'c_object_type_num'
    ]

    # Преобразуем указанные столбцы в числовой тип
    for col in columns_to_convert:
        if col in group.columns:
            group[col] = pd.to_numeric(group[col], errors='coerce')

    # Создаем лаги для всех указанных столбцов и собираем в DataFrame
    lagged_columns = {
        f'{col}_lag': group[col].shift(1) for col in columns_to_convert if col in group.columns
    }

    # Объединяем лаги с исходным DataFrame
    lagged_df = pd.concat([group, pd.DataFrame(lagged_columns)], axis=1)

    # Заполняем пропуски (NaN) значениями 0
    lagged_df = lagged_df.fillna(0)

    return lagged_df

# Применение функции для добавления признаков
train_data_with_features = train_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)
print(train_data_with_features.columns)

In [ ]:
#Classifier
def create_features(group):
    # Убедимся, что работаем с DataFrame и преобразуем столбцы в числовой тип
    group = group.copy()  # Создаем копию группы, чтобы избежать изменений в оригинальных данных
    columns_to_convert = [
        'time_to_tca','high_low_risk_CDM', 'max_risk_estimate', 'max_risk_scaling', 'mahalanobis_distance',
        'miss_distance', 'c_obs_used', 'relative_speed', 'relative_position_r', 'relative_position_t',
        'relative_position_n', 'relative_velocity_r', 'relative_velocity_t', 'relative_velocity_n',
        't_time_lastob_start', 't_time_lastob_end', 't_recommended_od_span', 't_actual_od_span',
        't_obs_available', 't_obs_used', 't_residuals_accepted', 't_weighted_rms', 't_rcs_estimate',
        't_cd_area_over_mass', 't_cr_area_over_mass', 't_sedr', 't_j2k_sma', 't_j2k_ecc', 't_j2k_inc',
        't_ct_r', 't_cn_r', 't_cn_t', 't_crdot_r', 't_crdot_t', 't_crdot_n', 't_ctdot_r', 't_ctdot_t',
        't_ctdot_n', 't_ctdot_rdot', 't_cndot_r', 't_cndot_t', 't_cndot_n', 't_cndot_rdot',
        't_cndot_tdot', 'c_time_lastob_start', 'c_time_lastob_end',
        'c_recommended_od_span','t_position_covariance_det_normalized','c_position_covariance_det_normalized',
        'c_actual_od_span', 'c_obs_available',
        'c_residuals_accepted', 'c_weighted_rms', 'c_rcs_estimate', 'c_cd_area_over_mass',
        'c_cr_area_over_mass', 'c_sedr', 'c_j2k_sma', 'c_j2k_ecc', 'c_j2k_inc', 'c_ct_r', 'c_cn_r',
        'c_cn_t', 'c_crdot_r', 'c_crdot_t', 'c_crdot_n', 'c_ctdot_r', 'c_ctdot_t', 'c_ctdot_n',
        'c_ctdot_rdot', 'c_cndot_r', 'c_cndot_t', 'c_cndot_n', 'c_cndot_rdot', 'c_cndot_tdot', 't_span',
        'c_span', 't_h_apo', 't_h_per', 'c_h_apo', 'c_h_per', 'geocentric_latitude', 'azimuth',
        'elevation', 't_sigma_r', 'c_sigma_r', 't_sigma_t', 'c_sigma_t', 't_sigma_n', 'c_sigma_n', 't_sigma_rdot',
        'c_sigma_rdot', 't_sigma_tdot', 'c_sigma_tdot', 't_sigma_ndot', 'c_sigma_ndot', 'F10', 'F3M',
        'SSN', 'AP', 'c_object_type_num'
    ]

    # Преобразуем указанные столбцы в числовой тип
    for col in columns_to_convert:
        if col in group.columns:
            group[col] = pd.to_numeric(group[col], errors='coerce')

    # Создаем лаги для всех указанных столбцов и собираем в DataFrame
    lagged_columns = {
        f'{col}_lag': group[col].shift(1) for col in columns_to_convert if col in group.columns
    }

    # Объединяем лаги с исходным DataFrame
    lagged_df = pd.concat([group, pd.DataFrame(lagged_columns)], axis=1)

    # Заполняем пропуски (NaN) значениями 0
    lagged_df = lagged_df.fillna(0)

    return lagged_df

# Применение функции для добавления признаков
train_data_with_features = train_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)
print(train_data_with_features.columns)

Index(['event_id', 'time_to_tca', 'mission_id', 'risk', 'max_risk_estimate',
       'max_risk_scaling', 'miss_distance', 'relative_speed',
       'relative_position_r', 'relative_position_t',
       ...
       'c_sigma_rdot_lag', 't_sigma_tdot_lag', 'c_sigma_tdot_lag',
       't_sigma_ndot_lag', 'c_sigma_ndot_lag', 'F10_lag', 'F3M_lag', 'SSN_lag',
       'AP_lag', 'c_object_type_num_lag'],
      dtype='object', length=208)


C:\Users\lofel\AppData\Local\Temp\ipykernel_18996\2876064805.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data_with_features = train_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)


In [ ]:
#Regressor
# Указание признаков
features = [
    'time_to_tca_lag', 'risk_lag', 'max_risk_estimate_lag', 'max_risk_scaling_lag', 'mahalanobis_distance_lag',
    'miss_distance_lag', 'c_obs_used_lag', 'relative_speed_lag', 'relative_position_r_lag', 'relative_position_t_lag',
    'relative_position_n_lag', 'relative_velocity_r_lag', 'relative_velocity_t_lag', 'relative_velocity_n_lag',
    't_time_lastob_start_lag', 't_time_lastob_end_lag', 't_recommended_od_span_lag', 't_actual_od_span_lag',
    't_obs_available_lag', 't_obs_used_lag', 't_residuals_accepted_lag', 't_weighted_rms_lag', 't_rcs_estimate_lag',
    't_cd_area_over_mass_lag', 't_cr_area_over_mass_lag', 't_sedr_lag', 't_j2k_sma_lag', 't_j2k_ecc_lag', 't_j2k_inc_lag',
    't_ct_r_lag', 't_cn_r_lag', 't_cn_t_lag', 't_crdot_r_lag', 't_crdot_t_lag', 't_crdot_n_lag', 't_ctdot_r_lag',
    't_ctdot_t_lag', 't_ctdot_n_lag', 't_ctdot_rdot_lag', 't_cndot_r_lag', 't_cndot_t_lag', 't_cndot_n_lag',
    't_cndot_rdot_lag', 't_cndot_tdot_lag', 'c_time_lastob_start_lag', 'c_time_lastob_end_lag',
    'c_recommended_od_span_lag', 'c_actual_od_span_lag', 'c_obs_available_lag', 'c_obs_used_lag',
    'c_residuals_accepted_lag','t_position_covariance_det_normalized_lag', 'c_position_covariance_det_normalized_lag',
    'c_weighted_rms_lag', 'c_rcs_estimate_lag', 'c_cd_area_over_mass_lag',
    'c_cr_area_over_mass_lag', 'c_sedr_lag', 'c_j2k_sma_lag', 'c_j2k_ecc_lag', 'c_j2k_inc_lag', 'c_ct_r_lag',
    'c_cn_r_lag', 'c_cn_t_lag', 'c_crdot_r_lag', 'c_crdot_t_lag', 'c_crdot_n_lag', 'c_ctdot_r_lag', 'c_ctdot_t_lag',
    'c_ctdot_n_lag', 'c_ctdot_rdot_lag', 'c_cndot_r_lag', 'c_cndot_t_lag', 'c_cndot_n_lag', 'c_cndot_rdot_lag',
    'c_cndot_tdot_lag', 't_span_lag', 'c_span_lag', 't_h_apo_lag', 't_h_per_lag', 'c_h_apo_lag', 'c_h_per_lag',
    'geocentric_latitude_lag', 'azimuth_lag', 'elevation_lag', 'mahalanobis_distance_lag',
    't_sigma_r_lag', 'c_sigma_r_lag', 't_sigma_t_lag', 'c_sigma_t_lag',
    't_sigma_n_lag', 'c_sigma_n_lag', 't_sigma_rdot_lag', 'c_sigma_rdot_lag', 't_sigma_tdot_lag', 'c_sigma_tdot_lag',
    't_sigma_ndot_lag', 'c_sigma_ndot_lag', 'F10_lag', 'F3M_lag', 'SSN_lag', 'AP_lag', 'c_object_type_num_lag'
]
# Подготовка данных для обучения (X_train и y_train)
X_train = train_data_with_features[features]  # Признаки для обучения
y_train = train_data_with_features['risk']  # Целевая переменная

# Заменяем NaN и inf значениями 0 (или другими значениями, если нужно)
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_train = X_train.fillna(0)  # Или можно использовать X_train.fillna(X_train.median())
X_train = X_train.clip(lower=-1e6, upper=1e6)
#Проверяем целевые данные (y_train), чтобы убедиться, что там нет NaN или inf
y_train = y_train.replace([np.inf, -np.inf], np.nan)
y_train = y_train.fillna(0)  # Или можно использовать y_train.fillna(y_train.median())

In [ ]:
#Classifier 1 lag
# Указание признаков
features = [
    'time_to_tca_lag', 'high_low_risk_CDM_lag', 'max_risk_estimate_lag', 'max_risk_scaling_lag', 'mahalanobis_distance_lag',
    'miss_distance_lag', 'c_obs_used_lag', 'relative_speed_lag', 'relative_position_r_lag', 'relative_position_t_lag',
    'relative_position_n_lag', 'relative_velocity_r_lag', 'relative_velocity_t_lag', 'relative_velocity_n_lag',
    't_time_lastob_start_lag', 't_time_lastob_end_lag', 't_recommended_od_span_lag', 't_actual_od_span_lag',
    't_obs_available_lag', 't_obs_used_lag', 't_residuals_accepted_lag', 't_weighted_rms_lag', 't_rcs_estimate_lag',
    't_cd_area_over_mass_lag', 't_cr_area_over_mass_lag', 't_sedr_lag', 't_j2k_sma_lag', 't_j2k_ecc_lag', 't_j2k_inc_lag',
    't_ct_r_lag', 't_cn_r_lag', 't_cn_t_lag', 't_crdot_r_lag', 't_crdot_t_lag', 't_crdot_n_lag', 't_ctdot_r_lag',
    't_ctdot_t_lag', 't_ctdot_n_lag', 't_ctdot_rdot_lag', 't_cndot_r_lag', 't_cndot_t_lag', 't_cndot_n_lag',
    't_cndot_rdot_lag', 't_cndot_tdot_lag', 'c_time_lastob_start_lag', 'c_time_lastob_end_lag',
    'c_recommended_od_span_lag', 'c_actual_od_span_lag', 'c_obs_available_lag', 'c_obs_used_lag',
    'c_residuals_accepted_lag','t_position_covariance_det_normalized_lag', 'c_position_covariance_det_normalized_lag',
    'c_weighted_rms_lag', 'c_rcs_estimate_lag', 'c_cd_area_over_mass_lag',
    'c_cr_area_over_mass_lag', 'c_sedr_lag', 'c_j2k_sma_lag', 'c_j2k_ecc_lag', 'c_j2k_inc_lag', 'c_ct_r_lag',
    'c_cn_r_lag', 'c_cn_t_lag', 'c_crdot_r_lag', 'c_crdot_t_lag', 'c_crdot_n_lag', 'c_ctdot_r_lag', 'c_ctdot_t_lag',
    'c_ctdot_n_lag', 'c_ctdot_rdot_lag', 'c_cndot_r_lag', 'c_cndot_t_lag', 'c_cndot_n_lag', 'c_cndot_rdot_lag',
    'c_cndot_tdot_lag', 't_span_lag', 'c_span_lag', 't_h_apo_lag', 't_h_per_lag', 'c_h_apo_lag', 'c_h_per_lag',
    'geocentric_latitude_lag', 'azimuth_lag', 'elevation_lag', 'mahalanobis_distance_lag',
    't_sigma_r_lag', 'c_sigma_r_lag', 't_sigma_t_lag', 'c_sigma_t_lag',
    't_sigma_n_lag', 'c_sigma_n_lag', 't_sigma_rdot_lag', 'c_sigma_rdot_lag', 't_sigma_tdot_lag', 'c_sigma_tdot_lag',
    't_sigma_ndot_lag', 'c_sigma_ndot_lag', 'F10_lag', 'F3M_lag', 'SSN_lag', 'AP_lag', 'c_object_type_num_lag'
]
# Подготовка данных для обучения (X_train и y_train)
X_train = train_data_with_features[features]  # Признаки для обучения
y_train = train_data_with_features['high_low_risk_CDM']  # Целевая переменная
# # Выделяем предпоследнюю строку для X_train и последнюю строку для y_train
# X_train = train_data_with_features.groupby('event_id').nth(-1)[features]  # последние строки
# y_train = train_data_with_features.groupby('event_id').nth(-1)['high_low_risk_CDM']  # Последние строки
# Заменяем NaN и inf значениями 0 (или другими значениями, если нужно)
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_train = X_train.fillna(0)  # Или можно использовать X_train.fillna(X_train.median())
X_train = X_train.clip(lower=-1e6, upper=1e6)
#Проверяем целевые данные (y_train), чтобы убедиться, что там нет NaN или inf
y_train = y_train.replace([np.inf, -np.inf], np.nan)
y_train = y_train.fillna(0)  # Или можно использовать y_train.fillna(y_train.median())

In [ ]:
# #Classifier lag > 1
# features = [
#         'time_to_tca','high_low_risk_CDM', 'max_risk_estimate', 'max_risk_scaling', 'mahalanobis_distance',
#         'miss_distance', 'c_obs_used', 'relative_speed', 'relative_position_r', 'relative_position_t',
#         'relative_position_n', 'relative_velocity_r', 'relative_velocity_t', 'relative_velocity_n',
#         't_time_lastob_start', 't_time_lastob_end', 't_recommended_od_span', 't_actual_od_span',
#         't_obs_available', 't_obs_used', 't_residuals_accepted', 't_weighted_rms', 't_rcs_estimate',
#         't_cd_area_over_mass', 't_cr_area_over_mass', 't_sedr', 't_j2k_sma', 't_j2k_ecc', 't_j2k_inc',
#         't_ct_r', 't_cn_r', 't_cn_t', 't_crdot_r', 't_crdot_t', 't_crdot_n', 't_ctdot_r', 't_ctdot_t',
#         't_ctdot_n', 't_ctdot_rdot', 't_cndot_r', 't_cndot_t', 't_cndot_n', 't_cndot_rdot',
#         't_cndot_tdot', 'c_time_lastob_start', 'c_time_lastob_end',
#         'c_recommended_od_span','t_position_covariance_det_normalized','c_position_covariance_det_normalized',
#         'c_actual_od_span', 'c_obs_available',
#         'c_residuals_accepted', 'c_weighted_rms', 'c_rcs_estimate', 'c_cd_area_over_mass',
#         'c_cr_area_over_mass', 'c_sedr', 'c_j2k_sma', 'c_j2k_ecc', 'c_j2k_inc', 'c_ct_r', 'c_cn_r',
#         'c_cn_t', 'c_crdot_r', 'c_crdot_t', 'c_crdot_n', 'c_ctdot_r', 'c_ctdot_t', 'c_ctdot_n',
#         'c_ctdot_rdot', 'c_cndot_r', 'c_cndot_t', 'c_cndot_n', 'c_cndot_rdot', 'c_cndot_tdot', 't_span',
#         'c_span', 't_h_apo', 't_h_per', 'c_h_apo', 'c_h_per', 'geocentric_latitude', 'azimuth',
#         'elevation', 't_sigma_r', 'c_sigma_r', 't_sigma_t', 'c_sigma_t', 't_sigma_n', 'c_sigma_n', 't_sigma_rdot',
#         'c_sigma_rdot', 't_sigma_tdot', 'c_sigma_tdot', 't_sigma_ndot', 'c_sigma_ndot', 'F10', 'F3M',
#         'SSN', 'AP', 'c_object_type_num'
#     ]
# def transform_for_catboost(data, features, lag=3):
#     X = []
#     y = []

#     grouped = data.groupby('event_id')

#     for event_id, group in grouped:
#         group = group[features + ['high_low_risk_CDM']].reset_index(drop=True)

#         # Создаем лаги
#         for i in range(len(group)):
#             if i < lag:
#                 # Если недостаточно данных, заполняем нулями
#                 lagged_features = np.pad(
#                     group.iloc[:i].values.flatten(),
#                     (lag * len(features) - len(group.iloc[:i].values.flatten()), 0),
#                     'constant'
#                 )
#             else:
#                 # Используем последние `lag` строк
#                 lagged_features = group.iloc[i-lag:i].values.flatten()

#             # Убедимся, что длина lagged_features соответствует ожидаемой
#             if len(lagged_features) == lag * len(features):
#                 X.append(lagged_features)
#                 y.append(group['high_low_risk_CDM'].iloc[i])

#     return np.array(X), np.array(y)

# # Преобразуем данные
# lag = 3  # Количество лагов
# X_train, y_train = transform_for_catboost(train_data_with_features, features, lag=lag)
# X_test, y_test = transform_for_catboost(test_data_with_features, features, lag=lag)

In [ ]:
# #Regressor lag > 1
# features = [
#         'time_to_tca','risk', 'max_risk_estimate', 'max_risk_scaling', 'mahalanobis_distance',
#         'miss_distance', 'c_obs_used', 'relative_speed', 'relative_position_r', 'relative_position_t',
#         'relative_position_n', 'relative_velocity_r', 'relative_velocity_t', 'relative_velocity_n',
#         't_time_lastob_start', 't_time_lastob_end', 't_recommended_od_span', 't_actual_od_span',
#         't_obs_available', 't_obs_used', 't_residuals_accepted', 't_weighted_rms', 't_rcs_estimate',
#         't_cd_area_over_mass', 't_cr_area_over_mass', 't_sedr', 't_j2k_sma', 't_j2k_ecc', 't_j2k_inc',
#         't_ct_r', 't_cn_r', 't_cn_t', 't_crdot_r', 't_crdot_t', 't_crdot_n', 't_ctdot_r', 't_ctdot_t',
#         't_ctdot_n', 't_ctdot_rdot', 't_cndot_r', 't_cndot_t', 't_cndot_n', 't_cndot_rdot',
#         't_cndot_tdot', 'c_time_lastob_start', 'c_time_lastob_end',
#         'c_recommended_od_span','t_position_covariance_det_normalized','c_position_covariance_det_normalized',
#         'c_actual_od_span', 'c_obs_available',
#         'c_residuals_accepted', 'c_weighted_rms', 'c_rcs_estimate', 'c_cd_area_over_mass',
#         'c_cr_area_over_mass', 'c_sedr', 'c_j2k_sma', 'c_j2k_ecc', 'c_j2k_inc', 'c_ct_r', 'c_cn_r',
#         'c_cn_t', 'c_crdot_r', 'c_crdot_t', 'c_crdot_n', 'c_ctdot_r', 'c_ctdot_t', 'c_ctdot_n',
#         'c_ctdot_rdot', 'c_cndot_r', 'c_cndot_t', 'c_cndot_n', 'c_cndot_rdot', 'c_cndot_tdot', 't_span',
#         'c_span', 't_h_apo', 't_h_per', 'c_h_apo', 'c_h_per', 'geocentric_latitude', 'azimuth',
#         'elevation', 't_sigma_r', 'c_sigma_r', 't_sigma_t', 'c_sigma_t', 't_sigma_n', 'c_sigma_n', 't_sigma_rdot',
#         'c_sigma_rdot', 't_sigma_tdot', 'c_sigma_tdot', 't_sigma_ndot', 'c_sigma_ndot', 'F10', 'F3M',
#         'SSN', 'AP', 'c_object_type_num'
#     ]
# def transform_for_catboost(data, features, lag=3):
#     X = []
#     y = []

#     grouped = data.groupby('event_id')

#     for event_id, group in grouped:
#         group = group[features + ['high_low_risk_CDM']].reset_index(drop=True)

#         # Создаем лаги
#         for i in range(len(group)):
#             if i < lag:
#                 # Если недостаточно данных, заполняем нулями
#                 lagged_features = np.pad(
#                     group.iloc[:i].values.flatten(),
#                     (lag * len(features) - len(group.iloc[:i].values.flatten()), 0),
#                     'constant'
#                 )
#             else:
#                 # Используем последние `lag` строк
#                 lagged_features = group.iloc[i-lag:i].values.flatten()
#             X.append(lagged_features)

#             # Целевое значение — риск текущей строки
#             y.append(group['risk'].iloc[i])

#     return np.array(X), np.array(y)

# # Преобразуем данные
# lag = 3  # Количество лагов
# X_train, y_train = transform_for_catboost(train_data_with_features, features, lag=lag)
# X_test, y_test = transform_for_catboost(test_data_with_features, features, lag=lag)

In [ ]:
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# # Создание пайплайна: масштабирование + модель
# pipeline = Pipeline([
#     ('scaler', StandardScaler()),  # Масштабирование данных
#     ('model', RandomForestRegressor(n_estimators=100, random_state=42))  # Модель
# ])

# # Настройка кросс-валидации
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Запуск кросс-валидации
# # Используем отрицательный MSE, так как cross_val_score максимизирует метрику
# cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# # Преобразуем отрицательный MSE в положительный
# mse_scores = -cv_scores

# # Анализ результатов
# print(f"Среднее MSE на кросс-валидации: {mse_scores.mean():.4f}")
# print(f"Стандартное отклонение MSE: {mse_scores.std():.4f}")

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# # Создание и обучение модели
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Создаем словарь весов классов
class_weights = {0: 1, 1: 1000000}

# Создание и обучение модели Random Forest
model = RandomForestClassifier(
    n_estimators=500,  # Количество деревьев
    max_depth=10,  # Максимальная глубина дерева
    random_state=42,  # Фиксируем результат для воспроизводимости
    class_weight=class_weights,  # Используем веса классов
    verbose=1  # Вывод информации во время обучения
)

# Обучение модели
model.fit(X_train_scaled, y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   23.5s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:  3.6min


RandomForestClassifier(class_weight={0: 1, 1: 1000000}, max_depth=10,
                       n_estimators=500, random_state=42, verbose=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Вычисление весов для регрессии (на основе распределения целевых значений)
# weights = np.where(y_train >= threshold, high_risk_weight, low_risk_weight)

# Создание и обучение модели RandomForestRegressor
model = RandomForestRegressor(
    n_estimators=100,      # Количество деревьев
    max_depth=10,        # Глубина деревьев (None означает неограниченная глубина)
    random_state=42        # Фиксация результата для воспроизводимости
)

# Обучение модели с учетом весов
model.fit(X_train_scaled, y_train)#, sample_weight=weights)

RandomForestRegressor(random_state=42)

In [ ]:
from catboost import CatBoostRegressor  # Для задачи регрессии
from sklearn.preprocessing import StandardScaler
import numpy as np

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Вычисление весов выборки
# Например, для high_risk объектов увеличиваем вес, low_risk оставляем меньше
sample_weight = np.where(y_train >= -6, 79, 0.5)

# Создание и обучение модели CatBoost для регрессии
model = CatBoostRegressor(
    iterations=500,           # Количество итераций
    learning_rate=0.1,        # Темп обучения
    depth=10,                 # Глубина деревьев
    random_seed=42,           # Фиксируем результат для воспроизводимости
    verbose=100               # Вывод информации каждые 100 итераций
)

# Обучение модели с учетом весов выборки
model.fit(
    X_train_scaled,
    y_train)

0:	learn: 9.1689872	total: 213ms	remaining: 1m 46s
100:	learn: 4.5740292	total: 15.2s	remaining: 59.9s
200:	learn: 4.3051016	total: 46.8s	remaining: 1m 9s
300:	learn: 4.1045274	total: 1m 24s	remaining: 56.1s
400:	learn: 3.9467593	total: 2m 11s	remaining: 32.4s
499:	learn: 3.8013801	total: 2m 37s	remaining: 0us


In [ ]:
#Catboost Classifier
from catboost import CatBoostClassifier  # Для классификации (если цель — бинарная классификация)
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Создание и обучение модели CatBoost
model = CatBoostClassifier(
    iterations=500,           # Количество итераций
    learning_rate=0.1,        # Темп обучения
    depth=10,                  # Глубина деревьев
    random_seed=42,           # Фиксируем результат для воспроизводимости
    verbose=100,              # Вывод информации каждые 100 итераций
    class_weights=[class_weights[0], class_weights[1]]  # Указание весов классов
)

model.fit(X_train_scaled, y_train)

0:	learn: 0.4175867	total: 154ms	remaining: 1m 16s
100:	learn: 0.0043450	total: 13.4s	remaining: 53.1s
200:	learn: 0.0042198	total: 30.7s	remaining: 45.7s
300:	learn: 0.0042198	total: 49.8s	remaining: 32.9s
400:	learn: 0.0042198	total: 1m 9s	remaining: 17.1s
499:	learn: 0.0042197	total: 1m 26s	remaining: 0us


In [ ]:
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Создание и обучение модели XGBoost
model = XGBRegressor(
    n_estimators=500,  # Количество деревьев
    learning_rate=0.1,  # Темп обучения
    max_depth=10,  # Глубина деревьев
    random_state=42,  # Фиксируем результат для воспроизводимости
    verbosity=1  # Минимальный вывод
)

# Обучение модели
model.fit(X_train_scaled, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


# Создание и обучение модели XGBoost
model = XGBClassifier(
    n_estimators=500,  # Количество деревьев
    learning_rate=0.1,  # Темп обучения
    max_depth=10,  # Глубина деревьев
    random_state=42,  # Фиксируем результат для воспроизводимости
    scale_pos_weight = 9600,  # Вес положительного класса
    verbosity=1  # Уровень детализации вывода
)

# Обучение модели
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Создание и обучение модели LightGBM
model = LGBMRegressor(
    n_estimators=500,  # Количество деревьев
    learning_rate=0.1,  # Темп обучения
    max_depth=10,  # Глубина деревьев
    num_leaves=1023,
    random_state=42,  # Фиксируем результат для воспроизводимости
    verbose=100  # Вывод информации каждые 100 итераций
)

# Обучение модели
model.fit(X_train_scaled, y_train)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.950002
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.046117
[LightGBM] [Debug] init for col-wise cost 0.001638 seconds, init for row-wise cost 0.013794 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23098
[LightGBM] [Info] Number of data points in the train set: 80743, number of used features: 103
[LightGBM] [Info] Start training from score -18.127910
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 361 and depth = 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 360 and depth = 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with

LGBMRegressor(max_depth=10, n_estimators=500, num_leaves=1023, random_state=42,
              verbose=100)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Создание и обучение модели LightGBM
model = LGBMClassifier(
    n_estimators=1000,  # Количество деревьев
    learning_rate=0.1,  # Темп обучения
    max_depth=10,  # Глубина деревьев
    random_state=42,  # Фиксируем результат для воспроизводимости
    class_weight={0: 1, 1: 111130},  # Весы классов
    verbose=1  # Уровень детализации вывода
)

# Обучение модели
model.fit(X_train_scaled, y_train)

[LightGBM] [Info] Number of positive: 6078, number of negative: 74665
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025802 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22846
[LightGBM] [Info] Number of data points in the train set: 80743, number of used features: 103
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.999889 -> initscore=9.110120
[LightGBM] [Info] Start training from score 9.110120


LGBMClassifier(class_weight={0: 1, 1: 111130}, max_depth=10, n_estimators=1000,
               random_state=42, verbose=1)

In [ ]:
# test_data_with_features = test_data_1


In [ ]:
#Regressor
# Применение функции для предсказания на тестовой выборке
test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)

# Подготовка данных для тестовой выборки
X_test = test_data_with_features[features]

# Заменяем NaN и inf значениями 0 (или другими значениями, если нужно)
X_test = X_test.replace([np.inf, -np.inf], np.nan)
X_test = X_test.fillna(0)  # Или можно использовать X_test.fillna(X_test.median())
X_test = X_test.clip(lower=-1e6, upper=1e6)

# Масштабируем данные (тестовые данные) на основе параметров, полученных из тренировочных данных
X_test_scaled = scaler.transform(X_test)  # Используем transform, а не fit_transform

# Применение функции для предсказания на отмасштабированных данных
def predict_last_row(group, model, features):
    # Применяем модель только к последней строке
    last_row = group.iloc[-1:]
    X_last = last_row[features].values.reshape(1, -1)  # Преобразуем в формат для модели

    # Предсказание для последней строки
    predicted_risk = model.predict(X_last)

    # Записываем предсказанное значение в столбец 'risk' для последней строки
    group.loc[group.index[-1], 'risk'] = predicted_risk[0]

    return group

# Применение функции для предсказания на тестовой выборке
test_data_with_predictions = test_data_with_features.groupby('event_id').apply(predict_last_row, model=model, features=features)

C:\Users\lofel\AppData\Local\Temp\ipykernel_18580\2409124148.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)
C:\Users\lofel\AppData\Local\Temp\ipykernel_18580\2409124148.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data_with_predictions = test_data_with_features.groupby('event_id').apply(

In [ ]:
#Classifier lag = 1
# Применение функции для предсказания на тестовой выборке
test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)

# Подготовка данных для тестовой выборки
X_test = test_data_with_features[features]

# Заменяем NaN и inf значениями 0 (или другими значениями, если нужно)
X_test = X_test.replace([np.inf, -np.inf], np.nan)
X_test = X_test.fillna(0)  # Или можно использовать X_test.fillna(X_test.median())
X_test = X_test.clip(lower=-1e6, upper=1e6)

# Масштабируем данные (тестовые данные) на основе параметров, полученных из тренировочных данных
X_test_scaled = scaler.transform(X_test)  # Используем transform, а не fit_transform

# Обновленная функция для предсказания на тестовой выборке
def predict_last_row(group, model, features):
    # Применяем модель только к последней строке
    last_row = group.iloc[-1:]
    X_last = last_row[features].values.reshape(1, -1)  # Преобразуем в формат для модели

    # Предсказание для последней строки (high_low_risk_CDM)
    predicted_class = model.predict(X_last)[0]

    # Записываем предсказанное значение в столбец 'high_low_risk_CDM' для последней строки
    group.loc[group.index[-1], 'high_low_risk_CDM'] = predicted_class

    return group

# Применение функции для предсказания на тестовой выборке
test_data_with_predictions = test_data_with_features.groupby('event_id').apply(predict_last_row, model=model, features=features)
test_data_with_features.head(16)

C:\Users\lofel\AppData\Local\Temp\ipykernel_18996\4151893000.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s


,event_id,time_to_tca,mission_id,risk,max_risk_estimate,max_risk_scaling,miss_distance,relative_speed,relative_position_r,relative_position_t,...,c_sigma_rdot_lag,t_sigma_tdot_lag,c_sigma_tdot_lag,t_sigma_ndot_lag,c_sigma_ndot_lag,F10_lag,F3M_lag,SSN_lag,AP_lag,c_object_type_num_lag
0,21,6.679228,3.0,-30.000000,-5.955068,77.647945,611.0,14713.0,-344.3,80.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,21,6.258407,3.0,-14.681728,-5.767512,24.752738,687.0,14713.0,-352.9,86.3,...,4.958491,0.003088,0.030325,0.008638,0.049432,84.0,83.0,52.0,2.0,0.0
2,21,5.974381,3.0,-30.000000,-5.734946,83.823880,556.0,14713.0,-331.3,65.8,...,4.935856,0.044480,0.030290,0.009603,0.049428,84.0,83.0,52.0,2.0,0.0
3,21,5.724197,3.0,-30.000000,-5.764977,113.335763,427.0,14713.0,-340.8,38.7,...,2.977488,0.001542,0.027189,0.004065,0.045996,82.0,83.0,38.0,1.0,0.0
4,21,5.337981,3.0,-30.000000,-5.766496,113.642555,426.0,14713.0,-341.0,34.6,...,2.671222,0.001446,0.024249,0.003434,0.043835,82.0,83.0,38.0,1.0,0.0
5,21,4.993201,3.0,-30.000000,-5.769807,113.264522,435.0,14713.0,-340.4,37.0,...,2.675425,0.001200,0.024247,0.002845,0.043835,82.0,83.0,38.0,1.0,0.0
6,21,4.705644,3.0,-30.000000,-5.602929,126.780672,387.0,14713.0,-322.0,30.9,...,2.704979,0.001162,0.024257,0.002776,0.043837,82.0,83.0,35.0,4.0,0.0
7,21,4.342530,3.0,-30.000000,-5.618163,129.237839,398.0,14713.0,-325.3,34.5,...,1.832818,0.001090,0.021387,0.002434,0.045797,82.0,83.0,35.0,4.0,0.0
8,21,4.069208,3.0,-30.000000,-5.548674,119.085313,320.0,14713.0,-316.5,-7.9,...,1.860791,0.001018,0.021413,0.002321,0.045798,82.0,83.0,35.0,4.0,0.0
9,21,3.648548,3.0,-30.000000,-5.593801,121.596400,325.0,14713.0,-320.0,5.0,...,1.684359,0.004383,0.021278,0.002301,0.045797,82.0,83.0,35.0,4.0,0.0


In [ ]:
#Naive solution
# Применение функции для предсказания на тестовой выборке
test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)

# Применение функции для предсказания
def predict_last_row(group, features):
    # Берём значение risk из предпоследней строки
    previous_risk = group['risk'].iloc[-2]

    # Записываем это значение в колонку risk последней строки
    group.loc[group.index[-1], 'risk'] = previous_risk

    return group

# Применение функции для предсказания на тестовой выборке
test_data_with_predictions = test_data_with_features.groupby('event_id').apply(predict_last_row, features=features)

C:\Users\lofel\AppData\Local\Temp\ipykernel_2352\3888423028.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)
C:\Users\lofel\AppData\Local\Temp\ipykernel_2352\3888423028.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data_with_predictions = test_data_with_features.groupby('event_id').apply(pr

In [ ]:
#Team Sesc
# Применение функции для предсказания на тестовой выборке
test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)

# Применение функции для предсказания
def predict_last_row(group, features):
    # Берём значение risk из предпоследней строки
    previous_risk = group['risk'].iloc[-2]
    previous_miss_disstance = group['miss_distance'].iloc[-2]
    if (previous_risk >= -6.04) and (previous_risk < -6.0):
        group.loc[group.index[-1], 'risk'] = -5.95
    if (previous_risk >= -6.4) and (previous_risk < -6.04):
        group.loc[group.index[-1], 'risk'] = -5.6
    if (previous_risk >= -7.3) and (previous_risk < -6.4):
        group.loc[group.index[-1], 'risk'] = -5
    if (previous_risk >= -4) and (previous_risk < -3.5):
        group.loc[group.index[-1], 'risk'] = -4
    if (previous_risk >= -3.5):
        group.loc[group.index[-1], 'risk'] = -3.5
    if (previous_miss_disstance > 30000):
        group.loc[group.index[-1], 'risk'] = -6.00001
    if (previous_risk < -7.3):
        group.loc[group.index[-1], 'risk'] = -30
    if (previous_risk >= -6) and (previous_risk < -4.0):
        group.loc[group.index[-1], 'risk'] = previous_risk
    return group

# Применение функции для предсказания на тестовой выборке
test_data_with_predictions = test_data_with_features.groupby('event_id').apply(predict_last_row, features=features)

C:\Users\lofel\AppData\Local\Temp\ipykernel_2352\312209632.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)
C:\Users\lofel\AppData\Local\Temp\ipykernel_2352\312209632.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data_with_predictions = test_data_with_features.groupby('event_id').apply(pred

In [ ]:
# # Обучение пайплайна на всех тренировочных данных
# pipeline.fit(X_train, y_train)

# # Применение функции для предсказания на тестовой выборке
# test_data_with_features = test_data_1.groupby('event_id').apply(create_features).reset_index(drop=True)

# # Подготовка данных для тестовой выборки
# X_test = test_data_with_features[features]

# # Заменяем NaN и inf значениями 0 (или другими значениями, если нужно)
# X_test = X_test.replace([np.inf, -np.inf], np.nan)
# X_test = X_test.fillna(0)  # Или можно использовать X_test.fillna(X_test.median())
# X_test = X_test.clip(lower=-1e6, upper=1e6)

# # Масштабируем данные (тестовые данные) на основе параметров, полученных из тренировочных данных
# X_test_scaled = pipeline.named_steps['scaler'].transform(X_test)  # Используем scaler из пайплайна

# # Применение функции для предсказания на отмасштабированных данных
# def predict_last_row(group, model, features):
#     # Применяем модель только к последней строке
#     last_row = group.iloc[-1:]
#     X_last = last_row[features].values.reshape(1, -1)  # Преобразуем в формат для модели

#     # Предсказание для последней строки
#     predicted_risk = model.predict(X_last)

#     # Записываем предсказанное значение в столбец 'risk' для последней строки
#     group.loc[group.index[-1], 'risk'] = predicted_risk[0]

#     return group

# # Применение функции для предсказания на тестовой выборке
# test_data_with_predictions = test_data_with_features.groupby('event_id').apply(
#     predict_last_row, model=pipeline.named_steps['model'], features=features
# )

# # Вывод результатов
# print(test_data_with_predictions)

In [ ]:
test_data_with_predictions.head(50)

event_id  time_to_tca  mission_id       risk  max_risk_estimate  \
event_id                                                                       
21       0         21     6.679228         3.0 -30.000000          -5.955068   
         1         21     6.258407         3.0 -14.681728          -5.767512   
         2         21     5.974381         3.0 -30.000000          -5.734946   
         3         21     5.724197         3.0 -30.000000          -5.764977   
         4         21     5.337981         3.0 -30.000000          -5.766496   
         5         21     4.993201         3.0 -30.000000          -5.769807   
         6         21     4.705644         3.0 -30.000000          -5.602929   
         7         21     4.342530         3.0 -30.000000          -5.618163   
         8         21     4.069208         3.0 -30.000000          -5.548674   
         9         21     3.648548         3.0 -30.000000          -5.593801   
         10        21     3.313444         3.0 -30.000000          -5.580871   
         11        21     3.006986         3.0 -30.000000          -5.557991   
         12        21     2.683814         3.0 -30.000000          -5.140381   
         13        21     2.378015         3.0 -30.000000          -5.145512   
         14        21     2.041105         3.0 -30.000000          -5.145512   
         15        21     0.000000         0.0   0.000000           0.000000   
27       16        27     6.623451         1.0  -6.670399          -5.091622   
         17        27     5.942896         1.0  -6.697236          -5.086451   
         18        27     5.551668         1.0 -30.000000          -4.912574   
         19        27     5.154167         1.0 -30.000000          -4.976748   
         20        27     4.906794         1.0 -30.000000          -4.985479   
         21        27     4.610000         1.0 -30.000000          -4.964570   
         22        27     4.210093         1.0 -30.000000          -4.952725   
         23        27     3.935730         1.0 -30.000000          -4.937794   
         24        27     3.678369         1.0 -30.000000          -4.936667   
         25        27     3.313080         1.0 -30.000000          -4.928855   
         26        27     3.044469         1.0 -30.000000          -4.924453   
         27        27     2.736228         1.0 -30.000000          -4.927750   
         28        27     2.367779         1.0 -30.000000          -4.923724   
         29        27     0.000000         0.0   0.000000           0.000000   
32       30        32     3.782006         1.0 -27.987585          -5.108909   
         31        32     3.442527         1.0 -28.065047          -5.090872   
         32        32     3.078060         1.0 -30.000000          -5.137451   
         33        32     2.848510         1.0 -30.000000          -5.164119   
         34        32     2.441740         1.0 -30.000000          -5.154654   
         35        32     2.106659         1.0 -30.000000          -5.151134   
         36        32     0.000000         0.0   0.000000           0.000000   
35       37        35     6.504201        15.0 -14.764724          -6.332174   
         38        35     5.518389        15.0 -23.423774          -6.229516   
         39        35     4.541538        15.0 -23.308300          -6.062482   
         40        35     3.532009        15.0 -30.000000          -6.499352   
         41        35     2.432261        15.0 -30.000000          -6.629117   
         42        35     0.000000         0.0   0.000000           0.000000   
42       43        42     6.867448         5.0 -27.646853          -6.501138   
         44        42     6.497240         5.0 -25.430275          -6.440452   
         45        42     6.239082         5.0 -24.464453          -6.393511   
         46        42     5.805900         5.0 -16.266081          -6.188626   
         47        42     5.536383         5.0 -14.287771          -6.044793   
         48  

In [ ]:
#Regressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Сбросим индекс в test_data_with_predictions без добавления его как столбца
test_data_with_predictions = test_data_with_predictions.reset_index(drop=True)
# Убедимся, что в main_data_set и test_data_with_predictions есть одинаковые event_id
# Получим последние строки для каждого event_id в main_data_set
main_data_last_rows = main_data_set.groupby('event_id').tail(1)

# Получаем предсказанные значения для последней строки каждого event_id
test_data_last_rows = test_data_with_predictions.groupby('event_id').tail(1)

# Объединяем два DataFrame, чтобы сравнить предсказания и фактические значения
comparison_df = pd.merge(main_data_last_rows[['event_id', 'risk']],
                         test_data_last_rows[['event_id', 'risk']],
                         on='event_id',
                         how='inner',
                         suffixes=('_actual', '_predicted'))

# Сравниваем фактические значения с предсказанными
print(comparison_df[['event_id', 'risk_actual', 'risk_predicted']])

# Рассчитываем метрики ошибки
mse = mean_squared_error(comparison_df['risk_actual'], comparison_df['risk_predicted'])
mae = mean_absolute_error(comparison_df['risk_actual'], comparison_df['risk_predicted'])
rmse = np.sqrt(mse)
# Выводим метрики
print(f"Root Mean Squared Error (RMSE): {rmse}")
comparison_df[['event_id', 'risk_actual', 'risk_predicted']].head(45)

      event_id  risk_actual  risk_predicted
0           21   -30.000000      -18.934506
1           27   -30.000000      -18.701004
2           32   -30.000000      -18.397536
3           35   -30.000000      -17.896337
4           42   -30.000000      -19.809237
...        ...          ...             ...
1653     13126   -30.000000      -18.568131
1654     13144   -30.000000      -17.988296
1655     13146   -28.147276      -18.108794
1656     13150   -30.000000      -20.012774
1657     13152   -30.000000      -20.024271

[1658 rows x 3 columns]
Mean Squared Error (MSE): 115.0431887531867
Mean Absolute Error (MAE): 10.476604395945916
Root Mean Squared Error (RMSE): 10.72581879173738


,event_id,risk_actual,risk_predicted
0,21,-30.000000,-18.934506
1,27,-30.000000,-18.701004
2,32,-30.000000,-18.397536
3,35,-30.000000,-17.896337
4,42,-30.000000,-19.809237
5,46,-30.000000,-17.713477
6,56,-6.991400,-17.895170
7,60,-30.000000,-19.968196
8,68,-30.000000,-17.583640
9,71,-30.000000,-19.629620


In [ ]:
#Classifier and Regressor
from sklearn.metrics import fbeta_score
main_data_set = main_data_set.assign(high_low_risk_CDM=np.where( (main_data_set['risk'] >= -6), 1, 0))
#Regressor
test_data_with_predictions = test_data_with_predictions.assign(high_low_risk_CDM=np.where( (test_data_with_predictions['risk'] >= -6), 1, 0))
# Сбросим индекс в test_data_with_predictions, чтобы event_id был обычным столбцом
test_data_with_predictions = test_data_with_predictions.reset_index(drop=True)
# Получим последние строки для каждого event_id в main_data_set
main_data_last_rows = main_data_set.groupby('event_id').tail(1)

# Получаем предсказанные значения для последней строки каждого event_id
test_data_last_rows = test_data_with_predictions.groupby('event_id').tail(1)
# Объединяем два DataFrame по event_id, чтобы гарантировать одинаковые event_id
comparison_df = pd.merge(main_data_last_rows[['event_id', 'high_low_risk_CDM']],
                         test_data_last_rows[['event_id', 'high_low_risk_CDM']],
                         on='event_id',
                         how='inner',
                         suffixes=('_actual', '_predicted'))

# Сравниваем фактические и предсказанные значения high_low_risk_CDM
y_true = comparison_df['high_low_risk_CDM_actual']
y_pred = comparison_df['high_low_risk_CDM_predicted']
# Подсчёт количества high_risk_event (значение 1) в y_pred
high_risk_count = y_pred.sum()

print(f"Количество предсказанных high_risk_event: {high_risk_count}")
# Вычисляем метрики
f2_score = fbeta_score(y_true, y_pred, beta=2)  # F2-score
# Выводим метрики
print(f"F2-score: {f2_score:.4f}")

Количество предсказанных high_risk_event: 0.0
Accuracy: 0.9855
Precision: 0.0000
Recall: 0.0000
F2-score: 0.0000
R2-score: -0.0147


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
